In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [2]:
from pathlib import Path

# Configurable parts
SOURCE = "dukascopy"
SYMBOL = "usdjpy"
MINUTES = 1
EVENT = '58m-dollar'
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Build base name
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"
RESAMPLED_NAME = f"{SOURCE}-{SYMBOL}-{EVENT}-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"

In [3]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
df.head()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,ret_mean_5,ret_mean_10
timestamp,,,,,,,,,,,,
2020-01-01 23:21:26.690,108.6435,108.6790,108.6435,108.6790,534460.000396,0.003709,13.189014,0.0360,0.000331,0.000331,-0.000064,-0.000071
2020-01-01 23:29:06.865,108.6790,108.7020,108.6635,108.7020,537749.999881,0.003010,13.195151,0.0230,0.000212,0.000212,0.000039,-0.000053
2020-01-01 23:36:19.371,108.7005,108.7020,108.6845,108.6880,535390.000820,0.003233,13.190753,-0.0140,-0.000129,-0.000129,-0.000003,-0.000064
2020-01-01 23:45:21.805,108.6880,108.6895,108.6760,108.6815,537089.997411,0.002859,13.193923,-0.0065,-0.000060,-0.000060,0.000117,-0.000063
2020-01-01 23:50:25.086,108.6805,108.7140,108.6800,108.7100,535309.998512,0.002339,13.190603,0.0285,0.000262,0.000262,0.000123,-0.000033


### Reset Timestamp into Column

In [4]:
df = df.reset_index()

# Techinical Indicator

In [5]:
import numpy as np
from ta.volatility import AverageTrueRange, BollingerBands, DonchianChannel
from ta.momentum import StochasticOscillator, RSIIndicator
from ta.trend import MACD, EMAIndicator, SMAIndicator

### EMA & SMA

In [6]:
# 5-period EMA
ema5 = EMAIndicator(close=df['close'], window=5)
df['ema5'] = ema5.ema_indicator()
ema5_slope = df['ema5'].diff()
df['ema5_slope'] = ema5_slope
# 20-period EMA
ema20 = EMAIndicator(close=df['close'], window=20)
df['ema20'] = ema20.ema_indicator()
ema20_slope = df['ema20'].diff()
df['ema20_slope'] = ema20_slope

### ATR

In [7]:
atr14 = AverageTrueRange(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=14
)
df['atr14'] = atr14.average_true_range()

atr20 = AverageTrueRange(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=20
)
df['atr20'] = atr20.average_true_range()
df['vol_adj_return'] = df['close_log_return'] / df['atr20']
df['close_to_atr'] = df['close_delta'] / df['atr20']

### BollingerBands

In [8]:
bb = BollingerBands(
    close=df['close'],
    window=20,
    window_dev=2
)
df['bb_upper'] = bb.bollinger_hband()
df['bb_lower'] = bb.bollinger_lband()
df['bb_mavg'] = bb.bollinger_mavg()
df['bb_width'] = df['bb_upper'] - df['bb_lower']
df['bb_position'] = (df['close'] - df['bb_lower']) / df['bb_width']

### Donchian Channel

In [9]:
dc = DonchianChannel(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=20
)
df['donchian_upper'] = dc.donchian_channel_hband()
df['donchian_lower'] = dc.donchian_channel_lband()
df['donchian_mid'] = dc.donchian_channel_mband()
df['donchian_width'] = df['donchian_upper'] - df['donchian_lower']

### Stochastic Oscillator

In [10]:
stoch = StochasticOscillator(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    window=14,
    smooth_window=3
)
df['stoch_k'] = stoch.stoch()
df['stoch_d'] = stoch.stoch_signal()

### RSI (Relative Strength Index)

In [11]:
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi14'] = rsi.rsi()


### MACD

In [12]:
macd = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_diff'] = macd.macd_diff()

# Time Features

In [13]:
df['unix_time'] = df['timestamp'].astype('int64') / 1e9

In [14]:
df['hour'] = df['timestamp'].dt.hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

In [15]:
df['dow'] = df['timestamp'].dt.dayofweek  # Monday=0
df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)

In [16]:
df['dom'] = df['timestamp'].dt.day
df['dom_sin'] = np.sin(2 * np.pi * df['dom'] / 31)
df['dom_cos'] = np.cos(2 * np.pi * df['dom'] / 31)

In [17]:
df['month'] = df['timestamp'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

## Drop NaN

In [18]:
df.isna().sum()

timestamp            0
open                 0
high                 0
low                  0
close                0
volume               0
spread               0
log_volume           0
close_delta          0
close_return         0
close_log_return     0
ret_mean_5           0
ret_mean_10          0
ema5                 4
ema5_slope           5
ema20               19
ema20_slope         20
atr14                0
atr20                0
vol_adj_return       0
close_to_atr         0
bb_upper            19
bb_lower            19
bb_mavg             19
bb_width            19
bb_position         20
donchian_upper      19
donchian_lower      19
donchian_mid        19
donchian_width      19
stoch_k             18
stoch_d             22
rsi14               13
macd                25
macd_signal         33
macd_diff           33
unix_time            0
hour                 0
hour_sin             0
hour_cos             0
dow                  0
dow_sin              0
dow_cos              0
dom        

In [19]:
df = df.dropna()
df.isna().sum()

timestamp           0
open                0
high                0
low                 0
close               0
volume              0
spread              0
log_volume          0
close_delta         0
close_return        0
close_log_return    0
ret_mean_5          0
ret_mean_10         0
ema5                0
ema5_slope          0
ema20               0
ema20_slope         0
atr14               0
atr20               0
vol_adj_return      0
close_to_atr        0
bb_upper            0
bb_lower            0
bb_mavg             0
bb_width            0
bb_position         0
donchian_upper      0
donchian_lower      0
donchian_mid        0
donchian_width      0
stoch_k             0
stoch_d             0
rsi14               0
macd                0
macd_signal         0
macd_diff           0
unix_time           0
hour                0
hour_sin            0
hour_cos            0
dow                 0
dow_sin             0
dow_cos             0
dom                 0
dom_sin             0
dom_cos   

In [20]:
df = df.set_index('timestamp')

In [21]:
df.head()

,open,high,low,close,volume,spread,log_volume,close_delta,close_return,close_log_return,...,hour_cos,dow,dow_sin,dow_cos,dom,dom_sin,dom_cos,month,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-02 03:35:29.827,108.6945,108.6995,108.6905,108.6955,533610.002041,0.002335,13.187422,0.001,0.000009,0.000009,...,0.707107,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 03:47:19.816,108.6960,108.7135,108.6895,108.6905,533940.002441,0.002247,13.188041,-0.005,-0.000046,-0.000046,...,0.707107,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:01:46.852,108.6900,108.7115,108.6870,108.7005,537560.003281,0.002303,13.194798,0.010,0.000092,0.000092,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:17:30.077,108.7000,108.7145,108.6990,108.7105,533880.000949,0.002420,13.187928,0.010,0.000092,0.000092,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025
2020-01-02 04:35:15.235,108.7100,108.7300,108.7080,108.7255,535220.003963,0.002204,13.190435,0.015,0.000138,0.000138,...,0.500000,3,0.433884,-0.900969,2,0.394356,0.918958,1,0.5,0.866025


In [22]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'log_volume',
       'close_delta', 'close_return', 'close_log_return', 'ret_mean_5',
       'ret_mean_10', 'ema5', 'ema5_slope', 'ema20', 'ema20_slope', 'atr14',
       'atr20', 'vol_adj_return', 'close_to_atr', 'bb_upper', 'bb_lower',
       'bb_mavg', 'bb_width', 'bb_position', 'donchian_upper',
       'donchian_lower', 'donchian_mid', 'donchian_width', 'stoch_k',
       'stoch_d', 'rsi14', 'macd', 'macd_signal', 'macd_diff', 'unix_time',
       'hour', 'hour_sin', 'hour_cos', 'dow', 'dow_sin', 'dow_cos', 'dom',
       'dom_sin', 'dom_cos', 'month', 'month_sin', 'month_cos'],
      dtype='object')

# Saving the file

In [23]:
df.to_pickle(PROCESSED_FILE_PATH)